dim_Address

In [0]:
%run "/Workspace/Utils/Utils"

In [0]:

from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, StringType, TimestampType, StructType, StructField
)

In [0]:
df = spark.table("adlslmcompany_silver.managed_silver.saleslt_address")

**Once the data has been cleaned, it should now be adapted to its final requirements, specifically as a dimension Address table.**

In [0]:

#Displaying DF

df.display()

In [0]:
def gold_clean_dimaddress(df): 

    #Drop columns that will not be necessary for this dimension table
    df= df.drop("rowguid", "ModifiedDate", "bronze_ingestion_timestamp", "silves_transformed_timestamp")

    # Add processed timestamp column
    df = df.withColumn("processed_timestamp", F.current_timestamp())

    #Cast to ensure datatype
    df = df.select(
         F.col('AddressID').cast(IntegerType()).alias('AddressID'),
         F.col('City').cast(StringType()).alias('City'), 
         F.col('StateProvince').cast(StringType()).alias('StateProvince'),
         F.col('CountryRegion').cast(StringType()).alias('CountryRegion'),
         F.col('processed_timestamp').cast(TimestampType()).alias('ModifiedDate'),
                 )
    
    return df


In [0]:

#Defining expected schema
expected_schema = StructType([
    StructField("AddressID", IntegerType(), False),             
    StructField("City", StringType(), True),                  
    StructField("StateProvince", StringType(), True),
    StructField("CountryRegion", StringType(), True),               
    StructField("processed_timestamp", TimestampType(), False) 
                            ])

In [0]:
gold_df = gold_clean_dimaddress(df)

In [0]:
#Comparing lenghts

compare_lengths(df, gold_df)

In [0]:
#Checking the schema 
_validate_schema(gold_df, expected_schema)

**IMPORTANT: Please note that this is a simulated project; the upsert operation will be executed within this notebook. In a production environment, a dedicated notebook containing only the function and validations would be developed. All function notebooks would be orchestrated by Azure Data Factory (ADF) pipelines or Azure Databricks (ADB) workflows. The method of upsert may vary based on the utilization of auto loader, streaming, or Change Data Feed (CDF).**

In [0]:
#Loading into Gold Layer

target_table= "dim_address"   

schema = "star_schema"

catalog = "adlscompany_gold"

primary_keys = ["AddressID"]


upsert_table(gold_df, target_table, primary_keys, schema, catalog )